In [1]:
import numpy as np
import errno
import os
from PIL import Image
import torch
import shutil
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.modules import Module
from torch.nn.modules.loss import _assert_no_grad
from tqdm import tqdm
tqdm.monitor_interval = 0

### Dataset
Download the OmniGlot dataset into a torch.Dataset. This will make it easy to batch load examples for training.

In [8]:
#Override __getitem__, __len__
cache = {}
class OmniglotDataset(data.Dataset):
    download_urls = [
        'https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip',
        'https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip'
    ]
    
    #copy the train/test/val splits from the Vinyals paper
    split_url = 'https://raw.githubusercontent.com/jakesnell/prototypical-networks/master/data/omniglot/splits/vinyals/'
    splits = {
        'test': split_url + 'test.txt',
        'train': split_url + 'train.txt',
        'trainval': split_url + 'trainval.txt',
        'val': split_url + 'val.txt',
    }
    splits_path = os.path.join('splits', 'vinyals')
    raw = 'raw'
    processed = 'data'

    def __init__(self, mode='train', root='../omniglot', download=True):
        '''
        @@mode: which of the sets to work with
        @@root: the directory where the dataset will be stored
        @@download: downloads the dataset
        '''
        super(OmniglotDataset, self).__init__()
        self.root = root

        if download:
            self.download()

        self.classes = self.get_current_classes(os.path.join(self.root, self.splits_path, mode + '.txt')) #for mode 
        self.items = self.find_items(os.path.join(self.root, self.processed), self.classes)
        self.idx_classes = self.index_classes(self.items)
        
        #y is a category label
        paths, self.y = zip(*[self.get_path_label(pl) for pl in range(len(self))]) 
        self.x = map(self.load_img, paths, range(len(paths))) #add the actual image to the Dataset object after rotation/resize
        self.x = list(self.x) #cast this to a list of images
    
    def __getitem__(self, idx):
        x = self.x[idx]
        return x, self.y[idx]

    def __len__(self):
        return len(self.items)
    
    
    def get_current_classes(self, fname):
        with open(fname) as f:
            classes = f.read().splitlines()
        return classes
    
    def find_items(self, root_dir, classes):
        '''
        returns a list of tuples with filename, label root, and its rotation
        '''
        retour = []
        #each image can have 4 rotations of the same image
        rots = ['/rot000', '/rot090', '/rot180', '/rot270']
        for (root, dirs, files) in os.walk(root_dir):
            for f in files:
                r = root.split('/')
                lr = len(r)
                label = r[lr - 2] + "/" + r[lr - 1]
                for rot in rots:
                    if label + rot in classes and (f.endswith("png")):
                        retour.extend([(f, label, root, rot)])
        print("== Dataset: Found %d items " % len(retour))
        return retour

    def index_classes(self, items):
        idx = {}
        for i in items:
            #if the image and rotation is not in the map, add it to the map (increasing in len)
            if (not i[1] + i[-1] in idx):  
                idx[i[1] + i[-1]] = len(idx)
        print("== Dataset: Found %d classes" % len(idx))
        return idx
    
    def get_path_label(self, index):
        filename = self.items[index][0]
        rot = self.items[index][-1]  
        img = str.join('/', [self.items[index][2], filename]) + rot
        target = self.idx_classes[self.items[index][1] + self.items[index][-1]]
        return img, target
    
    def _already_downloaded(self):
        return os.path.exists(os.path.join(self.root, self.processed))
    
    def download(self):
        '''
        Create and download the splits .txt
        Download raw zip'd omniglot data and unzip it. 
        '''
        import zipfile
        from six.moves import urllib

        if self._already_downloaded():
            return

        os.makedirs(os.path.join(self.root, self.splits_path))
        os.makedirs(os.path.join(self.root, self.raw))
        os.makedirs(os.path.join(self.root, self.processed))

        for k, url in self.splits.items():
            print('== Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[-1]
            file_path = os.path.join(self.root, self.splits_path, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())

        for url in self.download_urls:
            print('== Downloading ' + url)
            data = urllib.request.urlopen(url)
            filename = url.rpartition('/')[2]
            file_path = os.path.join(self.root, self.raw, filename)
            with open(file_path, 'wb') as f:
                f.write(data.read())
            orig_root = os.path.join(self.root, self.raw)
            print("== Unzip from " + file_path + " to " + orig_root)
            zip_ref = zipfile.ZipFile(file_path, 'r')
            zip_ref.extractall(orig_root)
            zip_ref.close()
            
        file_processed = os.path.join(self.root, self.processed)
        for p in ['images_background', 'images_evaluation']:
            for f in os.listdir(os.path.join(orig_root, p)):
                shutil.move(os.path.join(orig_root, p, f), file_processed)
            os.rmdir(os.path.join(orig_root, p))
        print("Download finished.")

    def load_img(self, path, idx):
        path, rot = path.split('/rot')
        if path in cache:
            x = cache[path]
        else:
            x = Image.open(path)
            cache[path] = x
        x = x.rotate(float(rot))   #rotate the image
        x = x.resize((28, 28))     #resize the image

        shape = 1, x.size[0], x.size[1]
        x = np.array(x, np.float32, copy=False)
        x = 1.0 - torch.from_numpy(x)
        x = x.transpose(0, 1).contiguous().view(shape)
        return x


In [9]:
x = OmniglotDataset()

== Dataset: Found 82240 items 
== Dataset: Found 4112 classes


### Data Sampler
A generator class for yielding a batch of indices with each training iteration.


In [10]:
class Sampler: 
    '''    
    Every iteration of the batch indices returns 'num_support' + 'num_query' samples
    for 'classes_per_it' random classes.
    
    __len__ is the number of iterations in an epoch.
    '''

    def __init__(self, labels, classes_per_it, num_samples, iterations):
        '''
        @@labels: an iterable containing all the labels for the current dataset
        @@classes_per_it: number of random classes per iteration
        @@num_samples: number of samples for each iteration for each class (support + query)
        @@iterations: number of iterations per epoch
        '''
        super(Sampler, self).__init__()
        self.labels = labels
        self.classes_per_it = classes_per_it
        self.sample_per_class = num_samples
        self.iterations = iterations

        self.classes, self.counts = np.unique(self.labels, return_counts=True)
        self.classes = torch.LongTensor(self.classes)

        self.idxs = range(len(self.labels))
        self.label_tens = np.empty((len(self.classes), max(self.counts)), dtype=int) * np.nan
        self.label_tens = torch.Tensor(self.label_tens)
        self.label_lens = torch.zeros_like(self.classes)
        for idx, label in enumerate(self.labels):
            label_idx = np.argwhere(self.classes == label)[0, 0]
            self.label_tens[label_idx, np.where(np.isnan(self.label_tens[label_idx]))[0][0]] = idx
            self.label_lens[label_idx] += 1

    def __iter__(self):
        '''
        yield a batch of indexes
        '''
        sample_iter= self.sample_per_class
        class_iter = self.classes_per_it

        for it in range(self.iterations):
            batch_size = sample_iter * class_iter
            batch = torch.LongTensor(batch_size)
            c_idxs = torch.randperm(len(self.classes))[:class_iter]
            for i, c in enumerate(self.classes[c_idxs]):
                s = slice(i * sample_iter, (i + 1) * sample_iter)
                label_idx = np.argwhere(self.classes == c)[0, 0]
                sample_idxs = torch.randperm(self.label_lens[label_idx])[:sample_iter]
                batch[s] = self.label_tens[label_idx][sample_idxs]
            batch = batch[torch.randperm(len(batch))]   # batch is a slice of the data mapping a file to its class label
            yield batch

    def __len__(self):
        '''
        returns the number of iterations per epoch
        '''
        return self.iterations

### Prototypical Architecture 
based off of https://arxiv.org/pdf/1703.05175.pdf which provide a clean alternative to matching networks